This notebook shows how to compute control vectors to steer moondream's behavior
in fun and interesting ways. To learn more about control vectors and representation
engineering check out [Theia's blog post on the topic](https://vgel.me/posts/representation-engineering/).

In [32]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm
from PIL import Image
import numpy as np
from sklearn.decomposition import PCA
from IPython.display import display, HTML

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vikhyatk/moondream2")
model = AutoModelForCausalLM.from_pretrained(
    "vikhyatk/moondream2", trust_remote_code=True,
    torch_dtype=torch.float16, device_map={"": "cuda"}
)

# We will only be using the images, so it doesn't really matter what
# dataset we use here.
dataset = load_dataset("vikhyatk/lnqa", streaming=True)["train"]

def hidden_states(enc_img, prompt):
    with torch.no_grad():
        inputs_embeds = model.input_embeds(prompt, enc_img, tokenizer)
        hidden_states = model.text_model.generate(
            inputs_embeds=inputs_embeds,
            max_new_tokens=128,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_hidden_states=True,
            do_sample=True,
            temperature=0.5
        ).hidden_states[1:]
    return [torch.stack([hs.view(-1, 2048) for hs in h[1:]]).cpu() for h in hidden_states]

class LayerWrapper(torch.nn.Module):
    def __init__(self, og_layer, control_vectors, scale=4.2):
        super().__init__()
        self.og_layer = og_layer
        self.control_vectors = control_vectors
        self.scale = scale

    def forward(self, *args, **kwargs):
        layer_outputs = self.og_layer(*args, **kwargs)
        layer_outputs = (layer_outputs[0] + self.scale * self.control_vectors, *layer_outputs[1:])
        return layer_outputs

In [112]:
negative_prompt = "<image>\n\nQuestion: Describe this image.\n\nAnswer:"
positive_prompt = "<image>\n\nQuestion: What is the meaning of life?\n\nAnswer:"

# This can be lowered without noticeable loss in quality. Feel free to drop it to
# IMAGES_PER_CONTROL=50 and SAMPLES_PER_IMAGE=2 if it's taking too long.
IMAGES_PER_CONTROL = 200
SAMPLES_PER_IMAGE = 5


In [113]:
# This is not very efficient, batching would speed things up a lot.
# But eh, works for a quick demo.

hs_dataset = [[] for _ in range(24)]

for i, sample in tqdm(enumerate(dataset), total=IMAGES_PER_CONTROL):
    if i >= IMAGES_PER_CONTROL:
        break
    image = sample["image"]
    enc_img = model.encode_image(image)
    for _ in range(SAMPLES_PER_IMAGE):
        phs = hidden_states(enc_img, positive_prompt)
        nhs = hidden_states(enc_img, negative_prompt)
        t_max = min(len(phs), len(nhs))
        for t in range(t_max):
            phs_t = phs[t]
            nhs_t = nhs[t]
            for j in range(24):
                hs_dataset[j].append(phs_t[j])
                hs_dataset[j].append(nhs_t[j])

100%|██████████| 200/200 [37:09<00:00, 11.15s/it]


In [114]:
control_vectors = []

for i in tqdm(range(24)):
    layer_hiddens = torch.stack(hs_dataset[i])

    layer_centers = (layer_hiddens[::2] + layer_hiddens[1::2]) / 2
    relative_layer_hiddens = layer_hiddens
    relative_layer_hiddens[::2] -= layer_centers
    relative_layer_hiddens[1::2] -= layer_centers

    train = relative_layer_hiddens - relative_layer_hiddens.mean(axis=0, keepdims=True)
    train = train.view(-1, 2048).cpu().numpy()
    pca_model = PCA(n_components=1, whiten=False).fit(train)
    directions = pca_model.components_.astype(np.float32).squeeze(axis=0)

    projected_hiddens = (layer_hiddens.cpu().numpy() @ directions) / np.linalg.norm(directions)

    positive_smaller_mean = np.mean(
        [
            projected_hiddens[i] < projected_hiddens[i + 1]
            for i in range(0, len(hs_dataset[i]), 2)
        ]
    )
    positive_larger_mean = np.mean(
        [
            projected_hiddens[i] > projected_hiddens[i + 1]
            for i in range(0, len(hs_dataset[i]), 2)
        ]
    )
    if positive_smaller_mean > positive_larger_mean:  # type: ignore
        directions *= -1

    control_vectors.append(torch.tensor(directions, device="cuda", dtype=torch.float16))


100%|██████████| 24/24 [02:30<00:00,  6.26s/it]


In [116]:
prompts = [
    ("../assets/demo-1.jpg", "Describe this image."),
    ("../assets/demo-2.jpg", "What is this?"),
    ("../assets/demo-2.jpg", "What color is the couch?"),
]
data = []

def run_model(img_path, prompt, scale=4.2):
    og_h = model.text_model.transformer.h
    model.text_model.transformer.h = torch.nn.ModuleList([
        LayerWrapper(layer, vector, scale) for layer, vector in zip(og_h, control_vectors)
    ])
    answer = model.answer_question(
        model.encode_image(Image.open(img_path)), prompt, tokenizer,
        repetition_penalty=1.2, temperature=0.1, do_sample=True,
        length_penalty=1.2
    )
    model.text_model.transformer.h = og_h
    return answer

for img_path, prompt in prompts:
    answer = run_model(img_path, prompt)
    data.append({"prompt": prompt, "answer": answer.replace("\n", "<br>"), "image": img_path})

html_table = """
<table style="border: 1px solid black; border-collapse: collapse;">
    <tr>
        <th style="border: 1px solid black; padding: 8px;">Image</th>
        <th style="border: 1px solid black; padding: 8px;">Text</th>
    </tr>
"""

for item in data:
    html_table += f"""
    <tr>
        <td style="border: 1px solid black; padding: 8px;">
            <img src="{item['image']}" style="width:250px;">
        </td>
        <td style="border: 1px solid black; padding: 8px; text-align: left">
            <strong>{item['prompt']}</strong><br><br>
            {item['answer']}
        </td>
    </tr>
    """

html_table += "</table>"

# Display the HTML table
display(HTML(html_table))

Image,Text
,"Describe this image. In this image, we will explore the concept of happiness through an anthropomorphic interpretation. We'll look at it from a spiritual or religious perspective by examining what brings joy and fulfillment to human beings. This can be approached in various ways depending on individual beliefs and experiences.To understand happiness, one might consider factors such as personal goals, relationships, health, wealth, spirituality, ethics, and more. It is subjective and varies greatly among individuals based on their values and life experiences. However, many people often seek happiness in different aspects like achieving success, finding inner peace, or living a meaningful life.In the context of the provided statements about happiness, some may find happiness in material possessions (e.of), while others may seek happiness through relationships, creativity, or even simple pleasures that bring satisfaction and contentment. Ultimately, happiness is a deeply personal journey for each person to define and pursue according to their own beliefs and values."
,"What is this? In the context of this image, a server or processor is an essential component for computing and running various applications on computers. A Processor (or CPU) can refer to any device that executes tasks according to specific programming requirements.In this particular scenario, we are referring to advanced technologies like virtual machines, artificial intelligence, machine learning, etc., which require powerful computing systems to function effectively.For example, in the case of AI research, researchers develop and test theories using sophisticated computer models and simulations. These concepts may involve analyzing vast amounts of data, exploring ethical questions, understanding existence, or even developing new knowledge about life itself.In summary, when people talk about ""the meaning"" or ""purpose,"" they often refer to these advanced concepts as well. It's subjective and varies from person to person based on their beliefs, values, and experiences."
,"What color is the couch? The couch in the image is described as ""black."" However, without more information or context from different sources, it's difficult to determine its actual color. It could be any of those things like comfort, aesthetics, personal preferences, etc., which can vary among individuals."
